In [ ]:
!pip install gymnasium

In [ ]:
import gymnasium as gym
#---#
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import IPython

#### Grid World에서의 배경

1. State: 각 격자 셀이 하나의 상태, 에이전트는 이러한 상태중 하나에 있을 수 있음.
2. Action: 에이전트는 현재 상태에서 다음상태로 이동하기 위해 상,하,좌,우 중 하나의 행동을 취할 수 있음.
3. Reward: 에이전트가 현재 상태에서 특정 action을 하면 얻어지는 보상.
4. Terminated: 하나의 에피소드가 종료되었음을 나타내는 상태

### 예비학습

#### 예시1

In [ ]:
action_space = gym.spaces.Discrete(4)
action_space

Discrete(4)

In [ ]:
[action_space.sample() for _ in range(4)] #이런식으로 뽑아서 볼 수 있음.

[0, 3, 1, 2]

#### 예시2

In [ ]:
state_space = gym.spaces.MultiDiscrete([4,4])
state_space

MultiDiscrete([4 4])

In [ ]:
[state_space.sample() for _ in range(4)]

[array([3, 1]), array([1, 3]), array([0, 0]), array([1, 1])]

약간의 디테일? 이번 강의에서 행동은 4개가 있고 에이전트는 4*4 의 상태를 가질 수 있다. 그래서 action_space와 state_space를 구분한 것..

In [ ]:
#시각화용 코드
def show(states):
    fig = plt.Figure()
    ax = fig.subplots()
    ax.matshow(np.zeros([4,4]), cmap='bwr',alpha=0.0)
    sc = ax.scatter(0, 0, color='red', s=500)
    ax.text(0, 0, 'start', ha='center', va='center')
    ax.text(3, 3, 'end', ha='center', va='center')
    # Adding grid lines to the plot
    ax.set_xticks(np.arange(-.5, 4, 1), minor=True)
    ax.set_yticks(np.arange(-.5, 4, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=2)
    state_space = gym.spaces.MultiDiscrete([4,4])
    def update(t):
        if states[t] in state_space:
            s1,s2 = states[t]
            states[t] = [s2,s1]
            sc.set_offsets(states[t])
        else:
            s1,s2 = states[t]
            s1 = s1 + 0.5 if s1 < 0 else (s1 - 0.5 if s1 > 3 else s1)
            s2 = s2 + 0.5 if s2 < 0 else (s2 - 0.5 if s2 > 3 else s2)
            states[t] = [s2,s1]
            sc.set_offsets(states[t])
    ani = FuncAnimation(fig,update,frames=len(states))
    display(IPython.display.HTML(ani.to_jshtml()))

In [ ]:
#사용 예시
show([[0,0],[0,1],[0,2],[0,3],[0,4]])
#그리드 밖에선 반만 걸치게 설계됨.

#### Env 클래스 구현

Env 클래스의 희망 작동 방식: (현재상태,action) -step-> (다음상태 ,보상)

Agent는 action을 env에게 주고, env는 다음 상태와 보상을 준다.

In [ ]:
action = 3
current_state = np.array([1,1]) #초기 위치

In [ ]:
action_to_direction = {
    0 : np.array([1, 0]), # row+, down
    1 : np.array([0, 1]), # col+, right
    2 : np.array([-1 ,0]), # row-, up
    3 : np.array([0, -1]) # col-, left
}
action_to_direction2 = {0: 'down', 1: 'right', 2: 'up', 3: 'left'} #자연어 버전

In [ ]:
action_to_direction[action] #액션을 받고 움직이는 방향을 반환

array([ 0, -1])

In [ ]:
next_state = current_state + action_to_direction[action]
next_state #다음 위치!

array([1, 0])

### Env 클래스 구현

In [ ]:
class GridWorld:
  def __init__(self):
    self.reset() #선언할 때 reset.
    self.state_space = gym.spaces.MultiDiscrete([4,4])
    self.action_space = gym.spaces.Discrete(4)
    self.action_to_direction = {
    0 : np.array([1, 0]), # row+, down
    1 : np.array([0, 1]), # col+, right
    2 : np.array([-1 ,0]), # row-, up
    3 : np.array([0, -1]) # col-, left
    }
    self.action_to_direction2 = {0: 'down', 1: 'right', 2: 'up', 3: 'left'} #자연어 버전

  def step(self, action):
    direction = self.action_to_direction[action]
    self.state = self.state + direction
    if self.state not in self.state_space:
      reward = -10
      terminated = True
    elif np.array_equal(self.state, np.array([3,3])):
      reward = 100
      terminated = True
    else:
      reward = -1
      terminated = False
    return self.state, reward, terminated
    #terminated = True일 때 마다 reset도 해줘야 할 거 같다.

  def reset(self):
    self.terminated = False
    self.state = np.array([0,0])
    return self.state

#클래스 만들 떄 self 체크 잘 하기!

In [ ]:
env = GridWorld()

In [ ]:
states = []
state = env.reset()
state #원점을 반환

array([0, 0])

In [ ]:
states.append(state) #상태를 모아두는 리스트

In [ ]:
states #이게 그냥 빈 리스트면 아래 반복문에서 문제가 생김..

[array([0, 0])]

In [ ]:
#여러번 시행하면서 움직이는 걸 관찰해보자
for _ in range(50):
  action = action_space.sample()
  state, reward, terminated = env.step(action)
  states.append(state)
  if terminated : break #True일 때 break

In [ ]:
states

[array([0, 0]), array([0, 1]), array([-1,  1])]

In [ ]:
show(states) #결과 시각화

### AgentRandom 클래스 설계

우리가 구현하고자 하는 AgentRandom의 기능:
- act() : action 결정(randomly)
- save_experience() : 데이터 저장
- learn() : 환경을 학습

In [ ]:
class AgentRandom:
  def __init__(self,env):
    #초기조건
    self.action_space = env.action_space #상식적으로 AgentRandom은 env와 상호작용하니까
    #env의 action_space를 받아주는 것이 적절.
    self.state_space = env.state_space
    self.n_episodes = 0
    self.n_experiences = 0
    #relay buffer 라고 한다.
    self.current_state = None
    self.action = None
    self.reward = None
    self.next_state = None
    self.terminated = None

    self.current_states = []
    self.actions = []
    self.rewards = []
    self.next_states = []
    self.terminations =[]

  def act(self):
    self.action = self.action_space.sample() #굳이 return하지 않는군..

  def learn(self):
    pass
  def save_experience(self):
    self.actions.append(self.action)
    self.current_states.append(self.current_state)
    self.rewards.append(self.rewards)
    self.next_states.append(self.next_state)
    self.terminations.append(self.terminated)
    self.n_experiences += 1

#### 작동 방식

In [ ]:
env = GridWorld()

In [ ]:
env.reset() #초기 설정.. 안하면 append가 안됨.

array([0, 0])

In [ ]:
agent = AgentRandom(env) #주어진 환경에 agent 객체 생성

In [ ]:
agent.act()

In [ ]:
agent.action #클래스 내부의 save_experience를 통해 저장됨.

2

In [ ]:
env.step(agent.action) #이러한 action이 환경에 전달

(array([-1,  0]), -10, True)

In [ ]:
#계속 진행만약 죽지 않았다면,
agent.act()
env.step(agent.action) #계속 진행

#### 여러번 반복하면서 관찰

In [ ]:
env = GridWorld()
agent = AgentRandom(env)

agent.current_state = env.reset()
#한번의 에피소드
for t in range(1000):
  # step1
  agent.act()
  # step2
  agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
  # step3
  agent.save_experience()
  agent.current_state = agent.next_state
  if agent.terminated:
    env.reset()
    break

In [ ]:
agent.current_states

[array([0, 0]), array([1, 0])]

In [ ]:
env = GridWorld()
agent = AgentRandom(env)


#열번의 에피소드
for i in range(10):
  agent.current_state = env.reset() #이거 안하면 절대 안됨..
  for t in range(1000):
    # step1
    agent.act()
    # step2
    agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
    # step3
    agent.save_experience()
    agent.current_state = agent.next_state
    if agent.terminated:
      env.reset()
      break

In [ ]:
agent.current_states

[array([0, 0]),
 array([0, 1]),
 array([0, 0]),
 array([1, 0]),
 array([0, 0]),
 array([1, 0]),
 array([2, 0]),
 array([3, 0]),
 array([0, 0]),
 array([0, 1]),
 array([0, 0]),
 array([0, 0]),
 array([0, 0]),
 array([0, 0]),
 array([0, 1]),
 array([0, 0]),
 array([1, 0]),
 array([0, 0]),
 array([1, 0]),
 array([1, 1]),
 array([2, 1]),
 array([3, 1]),
 array([0, 0]),
 array([1, 0]),
 array([0, 0]),
 array([1, 0]),
 array([1, 1]),
 array([1, 0]),
 array([0, 0])]

##### 짜증나는점... 빈 리스트면 저장(apeend)가 안됨

In [ ]:
env = GridWorld()
agent = AgentRandom(env)

In [ ]:
agent.save_experience()
agent.current_state

---

In [ ]:
np.concatenate([agent.next_states, agent.current_states],axis =1)[:10]

array([[ 0,  1,  0,  0],
       [-1,  1,  0,  1],
       [ 1,  0,  0,  0],
       [ 0,  0,  1,  0],
       [ 1,  0,  0,  0],
       [ 2,  0,  1,  0],
       [ 3,  0,  2,  0],
       [ 3, -1,  3,  0],
       [ 0,  1,  0,  0],
       [ 0,  0,  0,  1]])

---

#### 반복문 분석

In [ ]:
env = GridWorld()
agent = AgentRandom(env)

agent.current_state = env.reset()

In [ ]:
agent.next_states

[]

In [ ]:
agent.current_states

[]

In [ ]:
agent.act() #행동
agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
agent.save_experience() #저장

In [ ]:
agent.next_states

[array([0, 1])]

In [ ]:
agent.current_states

[array([0, 0])]

In [ ]:
agent.current_state = agent.next_states

---

### 환경과 상호작용

위에서 구현한 내용을 반복수에 따라 state를 보여주는 코드

#### 순서 암기!!

강화학습의 개괄적인 순서:
- Step1 : 에피소드 준비
- Step2 : 에피소드 진헹
  - step1 : 행동
  - step2 : 보상
  - step3 : 저장&학습
  - step4 : 다음 스텝 준비
- Step3 : 다음 에피소드 준비

In [ ]:
env = GridWorld()
agent = AgentRandom(env)

for _ in range(20):
  #Step1
  agent.current_state = env.reset() #무조건
  agent.terminated = False #생각해보니 terminated를 False로 바꿔주는 코드가 없네
  agent.score = 0
  #Step2
  for t in range(50):
    #step1
    agent.act()
    #step2
    agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
    #step3
    agent.save_experience()
    agent.learn
    #step4
    agent.current_state = agent.next_state
    if agent.terminated: break
  #Step3

In [ ]:
agent.next_states

[array([1, 0]),
 array([0, 0]),
 array([-1,  0]),
 array([ 0, -1]),
 array([0, 1]),
 array([1, 1]),
 array([1, 2]),
 array([0, 2]),
 array([-1,  2]),
 array([ 0, -1]),
 array([0, 1]),
 array([0, 0]),
 array([-1,  0]),
 array([1, 0]),
 array([ 1, -1]),
 array([-1,  0]),
 array([-1,  0]),
 array([1, 0]),
 array([2, 0]),
 array([ 2, -1]),
 array([1, 0]),
 array([2, 0]),
 array([2, 1]),
 array([3, 1]),
 array([2, 1]),
 array([1, 1]),
 array([2, 1]),
 array([2, 0]),
 array([ 2, -1]),
 array([ 0, -1]),
 array([-1,  0]),
 array([ 0, -1]),
 array([1, 0]),
 array([2, 0]),
 array([1, 0]),
 array([ 1, -1]),
 array([-1,  0]),
 array([0, 1]),
 array([0, 0]),
 array([1, 0]),
 array([2, 0]),
 array([ 2, -1]),
 array([-1,  0]),
 array([-1,  0]),
 array([ 0, -1]),
 array([1, 0]),
 array([2, 0]),
 array([2, 1]),
 array([3, 1]),
 array([3, 0]),
 array([ 3, -1])]

이렇게 보면... 에피소드별로 구분도 잘 안되고 성공여부도 잘 알 수가 없다..

#### 대안: score, playtimes를 활용

In [ ]:
class AgentRandom: #잡다하다..
    def __init__(self,env): #2. init에 env 입력.
        # define spaces
        self.action_space = env.action_space #1. 난 env가 뭔지 몰라
        self.state_space = env.state_space
        # other info
        self.score = 0
        self.scores = [] #에피소드별 점수 기록.
        self.playtimes = [] #게임별 플레이타임 기록.
        self.n_episodes = 0
        self.n_experiences = 0
        # replay_buffer (sarsa 에서 sars 가 들어감)
        self.current_state = None
        self.action = None
        self.reward = None
        self.next_state = None
        self.terminated = None
        #--# 핵심.
        self.current_states = []
        self.actions = []
        self.rewards = []
        self.next_states = []
        self.terminations = []
    def act(self):
        self.action = self.action_space.sample() #지금은 랜덤.
    def save_experience(self):
        self.actions.append(self.action)
        self.current_states.append(self.current_state)
        self.rewards.append(self.reward)
        self.next_states.append(self.next_state)
        self.terminations.append(self.terminated)
        #위는 환경이랑 상호작용하며 얻는 것들.
        #--#
        #아래는 agent의 편의상 기록하는 것들.
        self.n_experiences += 1 #n_ex = n_ex + 1 과 같은 코드
        self.score = self.score + self.reward

    def learn(self):
        pass

In [ ]:
env = GridWorld()
agent = AgentRandom(env)

#50번의 에피소드
for _ in range(50): #50번의 에피소드 관찰
  agent.current_state = env.reset()
  agent.score = 0
  for t in range(1000):
    # step1 행동
    agent.act()
    # step2 보상
    agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
    # step3 저장& 학습 (학습은 pass)
    agent.save_experience()
    agent.learn() #아직 구현 x
    # step4
    agent.current_state = agent.next_state
    if agent.terminated: #terminated == True 라면
      break
  agent.scores.append(agent.score) #agent.score는 한판 할 떄 reward
  agent.playtimes.append(t+1)

In [ ]:
#잡기술 게임 플레이 횟수, 보상점수, 시행 횟수
{i:[s,p] for i,(s,p) in enumerate(zip(agent.scores, agent.playtimes))}
#간간히 클리어 하는 경우가 보임

{0: [-10, 1],
 1: [-10, 1],
 2: [-11, 2],
 3: [-10, 1],
 4: [-11, 2],
 5: [-11, 2],
 6: [-10, 1],
 7: [-12, 3],
 8: [-13, 4],
 9: [-14, 5],
 10: [-11, 2],
 11: [-10, 1],
 12: [-11, 2],
 13: [-11, 2],
 14: [-16, 7],
 15: [-11, 2],
 16: [-10, 1],
 17: [-10, 1],
 18: [-10, 1],
 19: [-10, 1],
 20: [-11, 2],
 21: [-13, 4],
 22: [-19, 10],
 23: [-10, 1],
 24: [-10, 1],
 25: [-10, 1],
 26: [-19, 10],
 27: [-10, 1],
 28: [-30, 21],
 29: [-17, 8],
 30: [-15, 6],
 31: [-10, 1],
 32: [-10, 1],
 33: [-10, 1],
 34: [-10, 1],
 35: [-10, 1],
 36: [-10, 1],
 37: [-10, 1],
 38: [-10, 1],
 39: [-16, 7],
 40: [-10, 1],
 41: [-14, 5],
 42: [-18, 9],
 43: [-12, 3],
 44: [-20, 11],
 45: [-10, 1],
 46: [-12, 3],
 47: [-10, 1],
 48: [-20, 11],
 49: [-10, 1]}

### 최종 코드

최종 코드는 위에서 에피소드의 횟수를 기록하는 내용이 추가되었다.

In [ ]:
env = GridWorld()
agent = AgentRandom(env)

#50번의 에피소드
for _ in range(50): #50번의 에피소드 관찰
  agent.current_state = env.reset()
  agent.score = 0
  for t in range(1000):
    # step1 행동
    agent.act()
    # step2 보상
    agent.next_state, agent.reward, agent.terminated = env.step(agent.action)
    # step3 저장& 학습 (학습은 pass)
    agent.save_experience()
    agent.learn() #아직 구현 x
    # step4
    agent.current_state = agent.next_state
    if agent.terminated: #terminated == True 라면
      break
  agent.scores.append(agent.score) #agent.score는 한판 할 떄 reward
  agent.playtimes.append(t+1)

  agent.n_episodes += 1
  print(
        f"Epsiode: {agent.n_episodes} \t"
        f"Score: {agent.scores[-1]} \t"
        f"Playtime: {agent.playtimes[-1]}\t"
        f"경험: {agent.n_experiences}"
        )

Epsiode: 1 	Score: -10 	Playtime: 1	경험: 1
Epsiode: 2 	Score: -15 	Playtime: 6	경험: 7
Epsiode: 3 	Score: -10 	Playtime: 1	경험: 8
Epsiode: 4 	Score: -12 	Playtime: 3	경험: 11
Epsiode: 5 	Score: -11 	Playtime: 2	경험: 13
Epsiode: 6 	Score: -13 	Playtime: 4	경험: 17
Epsiode: 7 	Score: -10 	Playtime: 1	경험: 18
Epsiode: 8 	Score: -10 	Playtime: 1	경험: 19
Epsiode: 9 	Score: -14 	Playtime: 5	경험: 24
Epsiode: 10 	Score: -12 	Playtime: 3	경험: 27
Epsiode: 11 	Score: -10 	Playtime: 1	경험: 28
Epsiode: 12 	Score: -20 	Playtime: 11	경험: 39
Epsiode: 13 	Score: -10 	Playtime: 1	경험: 40
Epsiode: 14 	Score: -10 	Playtime: 1	경험: 41
Epsiode: 15 	Score: -20 	Playtime: 11	경험: 52
Epsiode: 16 	Score: -19 	Playtime: 10	경험: 62
Epsiode: 17 	Score: -10 	Playtime: 1	경험: 63
Epsiode: 18 	Score: -10 	Playtime: 1	경험: 64
Epsiode: 19 	Score: -11 	Playtime: 2	경험: 66
Epsiode: 20 	Score: -13 	Playtime: 4	경험: 70
Epsiode: 21 	Score: -10 	Playtime: 1	경험: 71
Epsiode: 22 	Score: -14 	Playtime: 5	경험: 76
Epsiode: 23 	Score: -10 	Playtime: 1	경험: 